# BioExplorer - Neurons soma report
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [1]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load neurons

In [2]:
assembly_name = 'Neurons'
load_morphologies = False
radius_multiplier = 1.0
sql_node_filter = 'guid%10=0'
if not load_morphologies:
    sql_node_filter=''
    radius_multiplier = 5.0

be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)
neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name='o1',
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SECTION,
    load_axon=False, load_basal_dendrites=load_morphologies, load_apical_dendrites=load_morphologies,
    simulation_report_id=0,
    radius_multiplier=radius_multiplier,
    sql_node_filter=sql_node_filter
)

In [3]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    cast_user_datas = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNAPSE, be.NEURON_MATERIAL_EFFERENT_SYNAPSE]:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(0.1)
            user_params.append(0.001)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(palette[0])
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(0.2)
            user_params.append(0.01)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_SHEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_DIFFUSE)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(0.1)
            user_params.append(0.1)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        cast_user_datas.append(True)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions, cast_user_datas=cast_user_datas
    )

In [4]:
model_ids = be.get_model_ids()['ids']
offset = model_ids[0]
set_morphology_materials(offset, 'Set3', be.NB_MATERIALS_PER_MORPHOLOGY)
core.set_renderer()

True

In [5]:
status = be.reset_camera()

In [6]:
status = core.set_renderer(
    background_color=[0,0,0],
    current='advanced', 
    samples_per_pixel=1,
    subsampling=4,
    max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1000.0
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.epsilon_multiplier = 1.0
params.max_ray_depth = 1
params.show_background = False
params.main_exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [7]:
status = core.set_renderer(
    background_color=[0,0,0],
    current='bio_explorer_voxel', 
    samples_per_pixel=1,
    subsampling=4,
    max_accum_frames=64)
params = core.BioExplorerVoxelRendererParams()
params.alpha_correction = 0.1
params.simulation_threshold = 0.3
params.main_exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [8]:
core.set_animation_parameters(current=0)

True

In [9]:
from bioexplorer import MovieMaker
from tqdm import tqdm

mm =MovieMaker(be)
frame = 0
for i in tqdm(range(0, 35000, 5)):
    core.set_animation_parameters(current=i)
    mm.create_snapshot(
        renderer='bio_explorer_voxel', base_name='%05d' % frame, path='/tmp',
        samples_per_pixel=4, size=[2160, 3840], show_progress=False,
    )
    frame += 1

  0%|          | 0/7000 [00:00<?, ?it/s]


KeyError: 'focusDistance'

In [ ]:
from bioexplorer import TransferFunction
model_ids = be.get_model_ids()['ids']
tf = TransferFunction(
    bioexplorer=be, model_id=model_ids[0],
    name='rainbow', size=9, alpha=0.5,
    value_range=[-80, -10])